In [1]:
import keras
import pandas as pd
import tensorflow as tf
from keras.layers import *
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
df = pd.read_csv("data/train_for_predict_adr.csv")
df = df.drop(["date"], axis=1)
df.head()

,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,...,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,1,0,257,27,0,2,1,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
1,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
2,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
3,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
4,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [3]:
X = df.drop(["adr"], axis=1)
y = df["adr"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1126)

In [6]:
earlystop = EarlyStopping(monitor='val_loss', patience=10)
reducelr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=0.001)

In [7]:
model = Sequential()
model.add(Dense(1024, input_dim=X_train.shape[1], kernel_initializer='glorot_uniform', activation='relu', kernel_regularizer="l1"))
model.add(Dropout(0.3))
model.add(Dense(1, kernel_initializer='glorot_uniform', activation='linear'))

model.compile(loss='mean_squared_error', optimizer='nadam')

In [8]:
model.fit(X_train, y_train, epochs=500, batch_size=128, validation_split=0.2, callbacks=[earlystop, reducelr])

Epoch 1/500
286/286 [==============================] - 1s 4ms/step - loss: 143004.6250 - val_loss: 23428.9121
Epoch 2/500
286/286 [==============================] - 1s 3ms/step - loss: 45326.1367 - val_loss: 2330.6956
Epoch 3/500
286/286 [==============================] - 1s 3ms/step - loss: 16646.5098 - val_loss: 19368.6758
Epoch 4/500
286/286 [==============================] - 1s 3ms/step - loss: 5131.5161 - val_loss: 1341.5270
Epoch 5/500
286/286 [==============================] - 1s 3ms/step - loss: 2992.4126 - val_loss: 959.4249
Epoch 6/500
286/286 [==============================] - 1s 3ms/step - loss: 2388.5173 - val_loss: 770.1861
Epoch 7/500
286/286 [==============================] - 1s 3ms/step - loss: 1360.0350 - val_loss: 732.4294
Epoch 8/500
286/286 [==============================] - 1s 3ms/step - loss: 1295.2028 - val_loss: 720.6419
Epoch 9/500
286/286 [==============================] - 1s 3ms/step - loss: 963.2286 - val_loss: 681.2436
Epoch 10/500
286/286 [===============

In [9]:
model.evaluate(X_test, y_test)

358/358 [==============================] - 0s 1ms/step - loss: 503.8035


503.803466796875

In [ ]:
# model.save("adr_model.h5")

In [ ]:
model.summary()

In [ ]:
model = keras.models.load_model("adr_model.h5")

In [ ]:
y_pred = pd.DataFrame(model.predict(X), columns=["adr"])
y_pred.shape

In [ ]:
y_pred.to_csv("train_adr_nn.csv", index=False)

### Predict for test data ADR

In [ ]:
df = pd.read_csv("test_for_predict_adr.csv")
df = df.drop(["date", "adr"], axis=1)
df.head()

In [ ]:
X = df.values
y_pred = pd.DataFrame(model.predict(X), columns=["adr"])

In [ ]:
y_pred.to_csv("test_adr_nn.csv", index=False)